In [20]:
import os
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from pivottablejs import pivot_ui
path = 'classifier_cv_results/'
file_names = os.listdir(path)
[file_name for file_name in file_names if 'labels' in file_name]

['San Bernardino - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Vegas - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Isla Vista - All Excerpts - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Marysville - All Excerpts - Final - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Newtown - All Excerpts - 1-2-2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Orlando - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Marysville - All Excerpts - Final - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Charleston - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Vegas - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Isla Vista - All Excerpts - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv',
 'Charleston - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv',
 'Newtown - All Excerpts - 1-2-2019.xlsx_sentences_tfidf_cv_results_lab

## Sentence Based Results For Each Event

In [10]:
sentence_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' in file_name and 'labels' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)
        event_df['event'] = file_name.split(' - ')[0]        
        sentence_results_df = pd.concat([sentence_results_df, event_df], ignore_index=True)

# get mean of cross val
df1 = sentence_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 
                                 'recall', 'precision', 'fscore',
                                 'num_test_exs', 'num_train_exs']]
df_cv_mean = df1.groupby(['label', 'event', 'vectorizer', 'classifier']).mean().reset_index()



San Bernardino - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Vegas - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Isla Vista - All Excerpts - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Orlando - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Marysville - All Excerpts - Final - 1_2_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Charleston - All Excerpts - 7_15_2019.xlsx_sentences_tfidf_cv_results_labels.csv
Newtown - All Excerpts - 1-2-2019.xlsx_sentences_tfidf_cv_results_labels.csv


In [4]:
# get max of each classification method
fmax_df = pd.DataFrame()
for grp in df_cv_mean.groupby(['label', 'event']):
    fmax_ids = grp[1].fscore == grp[1].fscore.max()
    fmax_row = pd.DataFrame(grp[1].loc[fmax_ids,:]).reset_index()
    if grp[1].fscore.max() >0:
        fmax_df = fmax_df.append(fmax_row.loc[0,:], ignore_index = True)
        
fmax_df

,classifier,event,fscore,index,label,num_test_exs,num_train_exs,precision,recall,vectorizer
0,svm_balanced,Charleston,0.398347,19.0,ACCOUNT,57.333333,210.666667,0.316495,0.537793,cust_all-3gram
1,svm_balanced,Isla Vista,0.767881,47.0,ACCOUNT,366.000000,1475.000000,0.714171,0.830418,char
2,svm_balanced,Orlando,0.444496,103.0,ACCOUNT,62.000000,253.000000,0.338007,0.650618,cust_only_alpha-1gram
3,logregcv_balanced,Vegas,0.280416,117.0,ACCOUNT,26.000000,92.000000,0.430403,0.210317,char
4,logregcv_balanced,Charleston,0.628657,153.0,EVENT,99.333333,413.666667,0.597270,0.664036,char
5,logregcv_balanced,Isla Vista,0.755741,189.0,EVENT,294.333333,1154.666667,0.729246,0.784463,char
6,logregcv_balanced,Orlando,0.666639,229.0,EVENT,81.000000,326.000000,0.636454,0.705084,cust_all-1gram
7,logregcv_balanced,Vegas,0.703499,261.0,EVENT,75.666667,307.333333,0.691133,0.717256,char
8,svm_balanced,Charleston,0.322888,319.0,GRIEF,20.666667,101.333333,0.229781,0.545238,cust_only_alpha-1gram
9,logregcv,Isla Vista,0.801695,332.0,GRIEF,152.000000,577.000000,0.861314,0.750255,char


In [5]:
df = fmax_df.loc[:, ['label', 'event', 'recall', 'precision', 'fscore',
                                 'num_test_exs', 'num_train_exs']]


labels = ['ALL']+list(set(df['label']))
events = ['ALL']+list(set(df['event']))
@interact
def show_results(column1='event', column2='label',
                 column3='num_test_exs', column4='num_train_exs',
                 event=events, label = labels, 
                 num_test = (0,50,1), num_train = (0,100,1)):
    
    df_len = df.shape[0]
    num_test_filt = df[column3] > num_test
    num_train_filt = df[column4] > num_train
    event_filt =[(df.loc[i, column1] == event) or (event == 'ALL') for i in range(df_len)]
    label_filt =[(df.loc[i, column2] == label) or (label == 'ALL') for i in range(df_len)]
    final_filt = [num_test_filt[i]&num_train_filt[i]&event_filt[i]&label_filt[i] for i in range(df_len)]
    
    return df.loc[final_filt]

interactive(children=(Text(value='event', description='column1'), Text(value='label', description='column2'), …

In [12]:
df = fmax_df.loc[:, ['label', 'event', 'recall', 'precision', 'fscore',
                                 'num_test_exs', 'num_train_exs']]

pivot_ui(df,outfile_path='sentences_pivottablejs.html')
#HTML(‘pivottablejs.html’)

## Excerpt Based Results for Each Event

In [15]:
excerpt_results_df = pd.DataFrame()

for file_name in file_names:
    if 'sentences' not in file_name and 'labels' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)
        event_df['event'] = file_name.split(' - ')[0]        
        excerpt_results_df = pd.concat([excerpt_results_df, event_df], ignore_index=True)

df = excerpt_results_df.loc[:, ['label', 'event', 'vectorizer', 'classifier', 
                                'recall', 'precision', 'fscore',
                                'num_test_exs', 'num_train_exs']]
df_cv_mean = df.groupby(['label', 'event', 'vectorizer', 'classifier']).mean()

Marysville - All Excerpts - Final - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Newtown - All Excerpts - 1-2-2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Charleston - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Vegas - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Isla Vista - All Excerpts - 1_2_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
Orlando - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv
San Bernardino - All Excerpts - 7_15_2019.xlsx_excerpts_tfidf_cv_results_labels.csv


In [16]:
# get max of each classification method
excerpt_fmax_df = pd.DataFrame()
for grp in df_cv_mean.groupby(['label', 'event']):
    fmax_ids = grp[1].fscore == grp[1].fscore.max()
    fmax_row = pd.DataFrame(grp[1].loc[fmax_ids,:]).reset_index()
    if grp[1].fscore.max() >0:
        excerpt_fmax_df = excerpt_fmax_df.append(fmax_row.loc[0,:], ignore_index = True)


In [17]:
df = excerpt_fmax_df.loc[:, ['label', 'event', 'recall', 'precision', 'fscore',
                                 'num_test_exs', 'num_train_exs']]

pivot_ui(df,outfile_path='excerpts_pivottablejs.html')


## All Events


### Sentence Based

In [ ]:
for file_name in file_names:
    if 'sentences' in file_name and 'labels' in file_name and 'fulldata' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)      
        #event_results_df = pd.concat([event_results_df, event_df], ignore_index=True)
        #print(event_df.columns)

df = event_df.loc[:, ['label', 'vectorizer', 'classifier', 'recall', 'precision', 'fscore']]

In [ ]:
df_cv_mean = df.groupby(['label', 'vectorizer', 'classifier']).mean()
# get max of each classification method
fmax_df = pd.DataFrame()
for grp in df_cv_mean.groupby(['label']):
    fmax_ids = grp[1].fscore == grp[1].fscore.max()
    fmax_row = pd.DataFrame(grp[1].loc[fmax_ids,:]).reset_index()
    if grp[1].fscore.max() >0:
        fmax_df = fmax_df.append(fmax_row.loc[0,:], ignore_index = True)
        
df = fmax_df.loc[:, ['label', 'recall', 'precision', 'fscore']]

#pivot_ui(df,outfile_path='fulldata_excerpts_pivottablejs.html')
df

### Excerpt Based

In [24]:
for file_name in file_names:
    if 'sentences' not in file_name and 'labels' in file_name and 'fulldata' in file_name:
        print(file_name)
        event_df = pd.read_csv(path+file_name)      
        #event_results_df = pd.concat([event_results_df, event_df], ignore_index=True)
        #print(event_df.columns)

df = event_df.loc[:, ['label', 'vectorizer', 'classifier', 'recall', 'precision', 'fscore']]

fulldata_excerpts_tfidf_cv_results_labels - fulldata_excerpts_tfidf_cv_results_labels.csv
Index(['Unnamed: 0', 'classifier', 'cv', 'fscore', 'label', 'num_test_exs',
       'num_train_exs', 'precision', 'recall', 'vectorizer'],
      dtype='object')


,label,vectorizer,classifier,recall,precision,fscore
0,HERO,3gram,svm_balanced,0.440000,0.687500,0.536585
1,HERO,1gram,svm_balanced,0.520000,0.722222,0.604651
2,HERO,char,svm_balanced,0.280000,0.700000,0.400000
3,RACECULTURE,3gram,svm_balanced,0.602564,0.474747,0.531073
4,RACECULTURE,1gram,svm_balanced,0.615385,0.400000,0.484848
5,RACECULTURE,char,svm_balanced,0.551282,0.483146,0.514970
6,RESOURCES,3gram,svm_balanced,0.722222,0.669118,0.694656
7,RESOURCES,1gram,svm_balanced,0.785714,0.585799,0.671186
8,RESOURCES,char,svm_balanced,0.738095,0.726562,0.732283
9,SAFETY,3gram,svm_balanced,0.763713,0.663004,0.709804


In [27]:
df_cv_mean = df.groupby(['label', 'vectorizer', 'classifier']).mean()
# get max of each classification method
fmax_df = pd.DataFrame()
for grp in df_cv_mean.groupby(['label']):
    fmax_ids = grp[1].fscore == grp[1].fscore.max()
    fmax_row = pd.DataFrame(grp[1].loc[fmax_ids,:]).reset_index()
    if grp[1].fscore.max() >0:
        fmax_df = fmax_df.append(fmax_row.loc[0,:], ignore_index = True)
        
df = fmax_df.loc[:, ['label', 'recall', 'precision', 'fscore']]

#pivot_ui(df,outfile_path='fulldata_excerpts_pivottablejs.html')
df

,classifier,fscore,label,precision,recall,vectorizer
0,svm_balanced,0.646427,ACCOUNT,0.600271,0.701298,char
1,svm_balanced,0.804812,EVENT,0.760016,0.855760,char
2,svm_balanced,0.710274,GRIEF,0.730314,0.691717,char
3,svm_balanced,0.623459,HERO,0.657407,0.601905,1gram
4,svm_balanced,0.646376,INVESTIGATION,0.600388,0.700388,3gram
5,svm_balanced,0.505202,JOURNEY,0.512640,0.499084,char
6,svm_balanced,0.607149,LEGAL,0.545679,0.684539,1gram
7,svm_balanced,0.543683,MEDIA,0.646586,0.469642,3gram
8,svm_balanced,0.153100,MISCELLANEOUS,0.358333,0.097963,char
9,svm_balanced,0.737217,MOURNING,0.729050,0.745644,char
